In [162]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',
    'EWY',
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ'
]

end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
max_leverage = 1
vol_target = 0.25

In [174]:
portfolio = PortfolioStandard(
    capital=169336.48958837058,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),        
    tc_model=TransactionCostFutu(), 
    systems=[        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOpt(gamma=10, hhi=0.2, lookback=60, confidence=1)], max_leverage=max_leverage, offset=60)
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

,Rebalanced Portfolio,Optimized Portfolio,^SPX,MVO - SMA1
Measure,,,,
Cumulative Return,1.856494,1.829670,1.174719,1.885328
Annualized Return,0.258031,0.252153,0.077397,0.262596
Annualized Volatility,0.215609,0.213927,0.180336,0.209446
Annualized Sharpe Ratio,1.040507,1.021213,0.242377,1.092920
Maximum Drawdown,-0.267453,-0.258937,-0.254251,-0.243479


In [165]:
portfolio.get_position_for_trade()

2024-08-27 01:17:18,802 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $169,336
2024-08-27 01:17:18,804 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $169,336


,AAPL,BRK-B,DXJ,EWY,META,NVDA,QQQ,SPY,TSLA
Date,,,,,,,,,
2022-01-03,311.0,125.0,99.0,23.0,0.0,686.0,25.0,52.0,47.0
2022-01-04,311.0,125.0,99.0,23.0,0.0,686.0,25.0,52.0,47.0
2022-01-05,311.0,125.0,99.0,23.0,0.0,686.0,25.0,52.0,47.0
2022-01-06,311.0,125.0,99.0,23.0,0.0,686.0,25.0,52.0,47.0
2022-01-07,311.0,125.0,99.0,23.0,0.0,686.0,25.0,52.0,47.0
...,...,...,...,...,...,...,...,...,...
2024-08-19,153.0,92.0,0.0,0.0,36.0,88.0,20.0,41.0,22.0
2024-08-20,153.0,92.0,0.0,0.0,36.0,88.0,20.0,41.0,22.0
2024-08-21,153.0,92.0,0.0,0.0,36.0,88.0,20.0,41.0,22.0


# Execute the Portfolio

In [168]:
# executor = ExecutorFutu(is_test=True)
# executor.set_portfolio(portfolio)
# orders = executor.execute(px_interval='5m')
# orders